In [1]:
import pandas as pd

df = pd.read_csv("../../data/clean/news_clean/filtered_news_dataset.csv")

In [2]:


assert {"company", "head_clean", "content_clean", "pubtime"}.issubset(df.columns), \
    "Expected columns company/head_clean/content_clean/pubtime are missing."

# If 'language' is missing, fill with default 'de'
if "language" not in df.columns:
    df["language"] = "de"

# Ensure 'pubtime' is converted to datetime, handle errors gracefully
df["pubtime"] = pd.to_datetime(df["pubtime"], errors="coerce")

# Combine headline + a short snippet of body text
SNIPPET_LEN = 200  # Set as reasonable default if not set elsewhere

def build_text(row):
    head = str(row.get("head_clean", "")).strip()
    body = str(row.get("content_clean", "")).strip()
    return (head + " " + body[:SNIPPET_LEN]).strip()

tmp = pd.DataFrame({
    "date":     df["pubtime"].dt.date,  # Will be NaT for unparsable dates
    "ticker":   df["company"].astype(str),
    "language": df["language"].astype(str),
    "text":     df.apply(build_text, axis=1).astype(str)
})

# Drop rows with missing date, ticker, or text, or very short texts
tmp = tmp.dropna(subset=["date", "ticker", "text"])
tmp = tmp[tmp["text"].str.len() > 3].copy()

print("Rows after cleaning:", len(tmp))
tmp.head(3)



Rows after cleaning: 242643


,date,ticker,language,text
0,2022-02-01,ABB,de,4-Wochenvorschau Schweiz Nachfolgend die wicht...
1,2024-02-23,ABB,de,Rosengren nimmt bei ABB den Hut und übergibt a...
2,2022-09-30,ABB,de,ABB trennt sich von restlichem Stromnetz-Gesch...


In [3]:
# drop unparsable dates if any slipped through
tmp = tmp.dropna(subset=["date"])

# normalize ticker format
tmp["ticker"] = tmp["ticker"].str.strip().str.upper()


In [4]:
print("Rows after cleaning:", len(tmp))

Rows after cleaning: 242643


In [7]:
from transformers import XLMRobertaTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# Force the slow SentencePiece tokenizer explicitly (avoids conversion path)
tok = XLMRobertaTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
mdl = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device).eval()

def score_texts(texts, max_len=256, batch_size=64):
    out = np.zeros(len(texts), dtype="float32")
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            enc = tok(
                texts[i:i+batch_size],
                return_tensors="pt",
                truncation=True, padding=True, max_length=max_len
            ).to(device)
            logits = mdl(**enc).logits.detach().cpu().numpy()
            probs = softmax(logits, axis=1)  # [neg, neu, pos]
            out[i:i+batch_size] = probs[:,2] - probs[:,0]
    return out



In [10]:
# --- Chunk 5: score all articles ---
import math

# optional: progress bar (comment out if you don't have tqdm)
try:
    from tqdm.auto import tqdm
    use_tqdm = True
except Exception:
    use_tqdm = False

texts = tmp["text"].tolist()
N = len(texts)
scores = np.zeros(N, dtype="float32")

batch = 64  # you can raise to 128 if you have plenty of GPU RAM
max_len = 256

mdl.eval()
with torch.no_grad():
    it = range(0, N, batch)
    if use_tqdm: it = tqdm(it, total=math.ceil(N/batch), desc="Scoring")
    for i in it:
        batch_texts = texts[i:i+batch]
        enc = tok(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_len
        ).to(device)
        logits = mdl(**enc).logits.detach().cpu().numpy()
        probs = softmax(logits, axis=1)   # [neg, neu, pos]
        scores[i:i+batch] = probs[:, 2] - probs[:, 0]  # pos - neg in [-1, 1]

tmp["sent_score"] = scores
import math

# quick sanity check
tmp[["date","ticker","language","sent_score"]].head(5)
tmp["sent_score"].describe()


# Save the result with sentiment scores to a regular folder (not notebook folder)
tmp.to_csv("../news_with_sentiment.csv", index=False)




Scoring:   0%|          | 0/3792 [00:00<?, ?it/s]

In [11]:
tmp.head()

,date,ticker,language,text,sent_score
0,2022-02-01,ABB,de,4-Wochenvorschau Schweiz Nachfolgend die wicht...,-0.000192
1,2024-02-23,ABB,de,Rosengren nimmt bei ABB den Hut und übergibt a...,0.052889
2,2022-09-30,ABB,de,ABB trennt sich von restlichem Stromnetz-Gesch...,-0.033073
3,2019-07-01,ABB,de,Schweizer Konzerne warten im EU-Börsenstreit e...,-0.344751
4,2020-06-15,ABB,de,+++Börsen-Ticker+++ - US-Börsen notieren deutl...,-0.080476


In [13]:
tmp['text'][4]







'+++Börsen-Ticker+++ - US-Börsen notieren deutlich tiefer - Alle Aktien des Dow Jones im Minus 16:50 Im Dow-Jones-Index , der 1,6 Prozent im Minus steht, notieren alle Aktien im roten Bereich. Am besten steht Walmart da, am schlechtesten Boeing :\n\n +++ 16:30 Die Novartis -Aktie steigt am Montag'

In [15]:
df['content_clean'][4]

'16:50 Im Dow-Jones-Index , der 1,6 Prozent im Minus steht, notieren alle Aktien im roten Bereich. Am besten steht Walmart da, am schlechtesten Boeing :\n\n +++ 16:30 Die Novartis -Aktie steigt am Montag um bis zu 1,4 Prozent, nachdem Citi die Anlageempfehlung von Neutral auf Kaufen erhöht und auf die Underperformance der Aktie gegenüber dem Gesundheitssektor im bisherigen Jahresverlauf verwiesen hat. Analysten um Andrew Baum sehen einen attraktiven Einstiegspunkt nach den Sicherheitsbedenken beim Augenmittel Beovu und relativem Mangel an Pipeline-Impulsgebern verglichen zu Wettbewerbern.\xa0Die Aktien sind seit Jahresbeginn 13 Prozent\xa0gefallen gegenüber minus\xa01,2 Prozent beim\xa0" Stoxx 600 Healthcare Index". Die Novartis -Aktie hat 19 Kaufempfehlungen, 9 Halten\xa0und 2 Verkaufen. Das durchschnittliche Kursziel lautet 96 Franken (derzeit: 81 Franken ).\n\n +++ 15:35 Der Dow Jones Industrial eröffnet zu Beginn der Woche bei\xa025’270 Punkten und einem Minus von mehr als 2,5 Proz

In [27]:
exp.head()

,date,ticker,language,text,sent_score,article_id,ticker,w,sent_weighted
0,2022-02-01,ABB,de,4-Wochenvorschau Schweiz Nachfolgend die wicht...,-0.000192,0,ABB,1.0,-0.000192
1,2024-02-23,ABB,de,Rosengren nimmt bei ABB den Hut und übergibt a...,0.052889,1,ABB,1.0,0.052889
2,2022-09-30,ABB,de,ABB trennt sich von restlichem Stromnetz-Gesch...,-0.033073,2,ABB,1.0,-0.033073
3,2019-07-01,ABB,de,Schweizer Konzerne warten im EU-Börsenstreit e...,-0.344751,3,ABB,1.0,-0.344751
4,2020-06-15,ABB,de,+++Börsen-Ticker+++ - US-Börsen notieren deutl...,-0.080476,4,ABB,1.0,-0.080476
